# Langchain integration

### LLMstudio setup

In [33]:
from llmstudio.llm.langchain import ChatLLMstudio
from llmstudio import LLM

In [38]:
# llm = ChatLLMstudio(model_id='openai/gpt-3.5-turbo', temperature=0)
llm = ChatLLMstudio(model_id='vertexai/gemini-1.5-flash', temperature=0)

### Langchain setup

In [37]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

In [35]:
llm.invoke('Hello')

AIMessage(content='Hello! \n\nHow can I help you today? \n', response_metadata={'token_usage': None, 'model_name': 'gemini-1.5-flash', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5945f8bd-7151-4d85-bf90-3bcc1eaabc6c-0')

### Example 1: Train ticket

In [39]:
@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"


def assistant(question: str)->str:
    tools = [get_departure, buy_ticket]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [40]:
assistant('When does my train depart? My ticket is 1234')


[StructuredTool(name='get_departure', description='Use this to fetch the departure time of a train', args_schema=<class 'pydantic.v1.main.get_departureSchema'>, func=<function get_departure at 0x107458b80>), StructuredTool(name='cancel_ticket', description='Use this to cancel a ticket', args_schema=<class 'pydantic.v1.main.cancel_ticketSchema'>, func=<function cancel_ticket at 0x135d5f4c0>), StructuredTool(name='buy_ticket', description='Use this to buy a ticket', args_schema=<class 'pydantic.v1.main.buy_ticketSchema'>, func=<function buy_ticket at 0x133045bc0>), StructuredTool(name='make_complaint', description='Use this to forward a complaint to the complaint department', args_schema=<class 'pydantic.v1.main.make_complaintSchema'>, func=<function make_complaint at 0x104345620>)]


> Entering new AgentExecutor chain...

Invoking: `get_departure` with `{'ticket_number': '1234'}`


12:00 AMYour train departs at 12:00 AM. 


> Finished chain.


{'input': 'When does my train depart? My ticket is 1234',
 'output': 'Your train departs at 12:00 AM. \n'}

In [41]:
assistant('Buy me a ticket to Madrid and tell the departure time')

[StructuredTool(name='get_departure', description='Use this to fetch the departure time of a train', args_schema=<class 'pydantic.v1.main.get_departureSchema'>, func=<function get_departure at 0x107458b80>), StructuredTool(name='cancel_ticket', description='Use this to cancel a ticket', args_schema=<class 'pydantic.v1.main.cancel_ticketSchema'>, func=<function cancel_ticket at 0x135d5f4c0>), StructuredTool(name='buy_ticket', description='Use this to buy a ticket', args_schema=<class 'pydantic.v1.main.buy_ticketSchema'>, func=<function buy_ticket at 0x133045bc0>), StructuredTool(name='make_complaint', description='Use this to forward a complaint to the complaint department', args_schema=<class 'pydantic.v1.main.make_complaintSchema'>, func=<function make_complaint at 0x104345620>)]


> Entering new AgentExecutor chain...

Invoking: `buy_ticket` with `{'destination': 'Madrid'}`


Bought ticket number 123456
Invoking: `get_departure` with `{'ticket_number': '123456'}`


12:00 AMYou have a

{'input': 'Buy me a ticket to Madrid and tell the departure time',
 'output': 'You have a ticket to Madrid with ticket number 123456. The departure time is 12:00 AM. \n'}

### Example 2: Start a party

In [43]:
@tool
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

@tool
def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."

@tool
def dim_lights(brightness: float) -> bool:
    """Dim the lights.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True


In [44]:
def assistant(question: str)->str:
    tools = [power_disco_ball, start_music, dim_lights]
    print(tools)

    #rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose = True, debug = True
    )

    response = agent_executor.invoke(
        {
            "input": question
        }
    )

    return response

In [45]:
assistant('Turn this into a party!')

[StructuredTool(name='power_disco_ball', description='Powers the spinning disco ball.', args_schema=<class 'pydantic.v1.main.power_disco_ballSchema'>, func=<function power_disco_ball at 0x1330468e0>), StructuredTool(name='start_music', description='Play some music matching the specified parameters.', args_schema=<class 'pydantic.v1.main.start_musicSchema'>, func=<function start_music at 0x133045a80>), StructuredTool(name='dim_lights', description='Dim the lights.', args_schema=<class 'pydantic.v1.main.dim_lightsSchema'>, func=<function dim_lights at 0x105a16980>)]


> Entering new AgentExecutor chain...

Invoking: `power_disco_ball` with `{'power': 1}`


Disco ball is spinning!
True
Invoking: `start_music` with `{'loud': 1, 'energetic': 1, 'bpm': 120}`


Starting music! energetic=True loud=True, bpm=120
Never gonna give you up.
Invoking: `power_disco_ball` with `{'power': 1}`


Disco ball is spinning!
True
Invoking: `dim_lights` with `{'brightness': 0.5}`


Lights are now set to 50%
Tr

{'input': 'Turn this into a party!',
 'output': 'Agent stopped due to iteration limit or time limit.'}